# Reinforcement Learning (Cross Entropy Method)

En esta práctica construiremos un sistema basado en aprendizaje por refuerzo para controlar el equilibrio de un péndulo invertido sobre una plataforma móvil, como muestra la animación inferior. Usaremos el framework de experimentación en RL <a href="https://gym.openai.com/">OpenAI Gym</a>, concretamente el entorno <a href="https://gym.openai.com/envs/CartPole-v0">Cart-pole-V0</a>. Y todo ello lo haremos con una simple neurona.

OpenAI Gym nos proporciona todo lo necesario para empezar: observación del estado del entorno, las acciones posibles a realizar y una representación gráfica de cómo va yendo el conjunto péndulo-plataforma.

<img src="imgs/cartpole.gif" width="50%">

Las observaciones que tenemos son: la posición de la plataforma, la velocidad de la plataforma, el ángulo del péndulo y la velocidad de rotación. Y las acciones son mover la plataforma a la derecha o a la izquierda.

In [1]:
import numpy as np
import gym

env = gym.make('CartPole-v0')

[2018-11-23 12:17:53,390] Making new env: CartPole-v0


Empezaremos ejecutando algunos ejemplos de interacción con el entorno elegiendo acciones al azar. Puedes descomentar la impresión de las observaciones para ver los datos.

In [ ]:
def run_random(env):
    for _ in range(20):
        observation = env.reset()
        for t in range(100):
            env.render()
            #print(observation)
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            #if done:
            #    print("Episode finished after {} timesteps".format(t+1))
            #    break
run_random(env)

Para controlar la plataforma utilizaremos una simple neurona que tendrá como entradas las observaciones y como salida la acción a aplicar: 0 para empujar en una dirección y 1 para la otra dirección. 

$$r = \sum_{i=1}^{4}observation_{i} \cdot w_i$$

Si $r<0$ entonces la salida de la neurona será $0$ y si $r\ge0$ la salida será $1$.

<img src="imgs/neuron.svg" width="60%">

Nuestro problema ahora es determinar el conjunto de pesos $W$ de la neurona que consigan mantener el péndulo en equilibrio. Y aquí es donde entra en juego el método de entropía cruzada (CEM). *A priori* no tenemos ningún criterio para establecer unos valores iniciales a los pesos, así que podemos asignarlos al azar. Lanzamos unos, por ejemplo, 100 episodios con  pesos aleatorios en cada episodio. Unos pesos funcionarán mejor que otros. Es decir, harán que el péndulo aguante más pasos en equilibiro antes de caerse. De esos 100 conjuntos de pesos escogemos los 10 mejores conjuntos. Con ellos generaremos una <a href="https://es.wikipedia.org/wiki/Distribuci%C3%B3n_normal">distribución gaussiana de probabilidad</a>.

$$ \mu = E[X] = \frac{\sum_{i=1}^{10}{x_i}}{10} $$

<br>

$$ \sigma = \sqrt{E[X - \mu]^2} = \sqrt{\frac{\sum_{i=1}^{10}{(x_i - \mu)^2}}{10}} $$

<br>

$$ f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} \cdot e^{\frac{-(x-\mu)^2}{2\sigma^2}}$$

A partir de esta función generaremos una nueva lista de conjuntos de pesos $(x_1, x_2, \dots x_{100}) \sim N(\mu, \sigma)$. La diferencia es que ahora este nuevo conjunto estará compuesto por mejores soluciones que la lista anterior. Si repetimos esta operación un número de veces llegaremos a buenas soluciones que logren el objetivo buscado.

In [3]:
mu = [0., 0., 0., 0.]  # first means
sigma = [1., 1., 1., 1.]  # first standard deviations
episodies = 100
iterations = 10

Definimos la función que ejecutará cada episodio (desde el momento inicial hasta que llegamos a los `max_reward` pasos o hasta que el péndulo se nos caiga.)

In [4]:
def run_episode(env, weights, render=False, max_reward=200):
    observation = env.reset()
    totalreward = 0
    for _ in range(max_reward):
        if render:
            env.render()
        action = 0 if np.matmul(weights, observation) < 0 else 1  # this line is our agent (just one neuron!!)
        observation, reward, done, info = env.step(action)
        totalreward += reward
        if done:
            break
    return totalreward

Comenzamos las iteraciones del método de entropía cruzada. 

In [5]:
for iteration in range(iterations):

    weights = []
    for m, s in zip(mu, sigma):
        weights.append(np.random.normal(m, s, episodies))
    weights = np.transpose(weights)

    rewards = []
    number_of_goals = 0

    for w in weights:
        r = run_episode(env, w, max_reward=500)
        rewards.append(r)
        if r == 500:
            number_of_goals += 1

    # We combine in a list parameteres+rewards and sort it by rewards.
    # To do that we use a lambda function
    l = sorted(zip(weights, rewards), key=lambda pair: pair[1])
    # We get the last ten (they will be those with the higher reward), but
    # only first component (parameters) is needed. 
    l = list(zip(*l[-10:])[0])

    mu = np.mean(l, 0)
    sigma = np.std(l, 0)

    print("------------")
    print("Iteration:", iteration)
    print("Mean:", mu)
    print("Standard deviation:", sigma)
    print("# goals:", number_of_goals)

------------
('Iteration:', 0)
('Mean:', array([ 0.22622696,  0.00262583,  0.62822957,  1.38080648]))
('Standard deviation:', array([ 0.81368334,  0.63275975,  0.661636  ,  0.74218545]))
('# goals:', 2)
------------
('Iteration:', 1)
('Mean:', array([-0.03691883,  0.52496577,  0.583289  ,  1.65092701]))
('Standard deviation:', array([ 0.15015612,  0.42542505,  0.3516454 ,  0.58044865]))
('# goals:', 13)
------------
('Iteration:', 2)
('Mean:', array([-0.09060018,  0.53560059,  0.8367906 ,  1.64534895]))
('Standard deviation:', array([ 0.0955591 ,  0.34067951,  0.28141821,  0.57870654]))
('# goals:', 62)
------------
('Iteration:', 3)
('Mean:', array([-0.07337055,  0.53950002,  0.8171924 ,  1.42793988]))
('Standard deviation:', array([ 0.06921299,  0.25670751,  0.22083462,  0.49368293]))
('# goals:', 84)
------------
('Iteration:', 4)
('Mean:', array([-0.06388024,  0.58374714,  0.79253672,  1.43311962]))
('Standard deviation:', array([ 0.03696111,  0.20729629,  0.26922627,  0.69427696])

Ejecutamos un episodio con 1500 pasos a ver qué tal va.

In [6]:
run_episode(env, mu, render=True, max_reward=1500)

1399.0